# Converting efficientdet from Tensorflow to ONNX

Google recently [published the new object detection model efficientdet](
https://github.com/google/automl/tree/master/efficientdet) that show great performance and accuracy. The models are found [here](https://github.com/google/automl/tree/master/efficientdet). 
This tutorial shows how to convert it to ONNX.

To start, we setup a few environment variables, download source and the pre-trained checkpoint.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import os
import time

%cd /tmp

HOME = os.getcwd()
MODEL = "efficientdet-d0"
os.environ['PYTHONPATH'] = os.path.join(HOME, "tpu")
os.environ['MODEL'] = MODEL
os.environ['CUDA_VISIBLE_DEVICES'] = ""

/tmp


In [ ]:
# download the code. At the time of this writing the last
# commit is 6ecb218626da627ca4c4e4bed4b760c7e66a1b70.
%cd {HOME}
!git clone https://github.com/google/automl

In [ ]:
%cd {HOME}/automl/efficientdet

In [ ]:
!wget -q https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/$MODEL.tar.gz
!tar -C /tmp -zxf $MODEL.tar.gz
!wget -O img.png  -q 'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'

In [3]:
%cd /mnt/sdb/home/tf/tensorflow/automl/efficientdet

/mnt/sdb/home/tf/tensorflow/automl/efficientdet


In [4]:
import tensorflow as tf
import inference
from inference import *
import PIL
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import math
import matplotlib.pyplot as plt

Read the checkpoint and export the final saved-model. We use existing code but need to make a small change in the code:

In automl/efficientdet/anchors.py find the code with:
```
top_detections_cls = tf.stack([top_detections_cls[:, 0] * image_scale,
                               top_detections_cls[:, 1] * image_scale,
                               height * image_scale, width * image_scale,
                               top_detections_cls[:, 4]], axis=-1)
```
and change axis from -1 to 1.

Then find
```
if disable_pyfun:
  return _generate_detections_tf(
      cls_outputs,
      box_outputs,
      self._anchors.boxes,
      indices,
      classes,
      image_id,
      image_scale,
      self._num_classes,
      min_score_thresh=min_score_thresh,
      max_boxes_to_draw=max_boxes_to_draw,
      use_native_nms=False
  )
```
and change use_native_nms from False to True.

Now run the code below to create the final saved-model:

In [5]:
class Driver(ServingDriver):
    def __init__(self, model_name, ckpt_path, image_size,
                 batch_size=1, num_classes=None, label_id_mapping=None):
        super(Driver, self).__init__(model_name, ckpt_path, image_size, batch_size, num_classes, label_id_mapping)
        self.disable_pyfun = True

    def export(self, output_dir):
        """Export a saved model."""
        signitures = self.signitures
        signature_def_map = {
            'serving_default':
                tf.saved_model.predict_signature_def(
                    {signitures['image_arrays'].name: signitures['image_arrays']},
                    {signitures['prediction'].name: signitures['prediction']}),
        }
        b = tf.saved_model.Builder(output_dir)
        b.add_meta_graph_and_variables(
            self.sess,
            tags=['serve'],
            signature_def_map=signature_def_map,
            assets_collection=tf.get_collection(tf.GraphKeys.ASSET_FILEPATHS),
            clear_devices=True)
        b.save()
        logging.info('Model saved at %s', output_dir)

In [ ]:
imgs = []
for f in ["img.png"]:
    imgs.append(np.array(PIL.Image.open(f)))

tf.compat.v1.disable_eager_execution()
output = os.path.join('.', MODEL)
if tf.io.gfile.exists(output):
    tf.io.gfile.rmtree(output)

driver = Driver(MODEL, os.path.join('/tmp', MODEL), image_size=512, batch_size=len(imgs))
driver.build()
driver.export(output)

We can convert to ONNX directly from the Install and run tf2onnx directly on the saved-model:

In [ ]:
!pip install git+https://github.com/onnx/tensorflow-onnx
!pip install onnxruntime

In [ ]:
!python -m tf2onnx.convert --opset 11 --saved-model $MODEL --output $MODEL.onnx

In [ ]:
!wget -O img.png  -q 'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'

Now that we have the ONNX model we can write a quick demo using tensorflow and onnxruntime.
The model returns an numpy array of shape [batch,detections,7]. detections are limitited to 50.
The 7 float32 retuned for each detection are
```
box = detection[1:5], as ymin, xmin, ymax, xmax
                (xmax and ymax are offsets to xmin and ymin)
classes = detection[6], the coco label
scores = detection[5], the score, values above 0.2 are probably good detections
```
Some functions to draw the bounding boxes:

In [5]:
img = Image.open("img.png")
plt.axis('off')
plt.imshow(img)
plt.show()

# we feed a batch of images. Because the model can be any image size and numpy 
# can't handle such batches we keep the batch size at 1.
images = np.array([np.asarray(img, dtype='uint8')])

coco_classes = {
    1: 'person',
    2: 'bicycle',
    3: 'car',
    4: 'motorcycle',
    5: 'airplane',
    6: 'bus',
    7: 'train',
    8: 'truck',
    9: 'boat',
    10: 'traffic light',
}


def draw_detection(draw, d, c):
    """Draw box and label for 1 detection."""
    ymin, xmin, ymax, xmax = d
    xmax += xmin
    ymax += ymin
    label = coco_classes[c]
    # print(ymin, xmin, ymax, xmax, label)
    label_size = draw.textsize(label)
    if ymax - label_size[1] >= 0:
        text_origin = tuple(np.array([xmin, ymax - label_size[1]]))
    else:
        text_origin = tuple(np.array([xmin, ymax + 1]))
    color = ImageColor.getrgb("red")
    thickness = 0
    draw.rectangle([xmin + thickness, ymax + thickness, xmax - thickness, ymin - thickness], outline=color)
    draw.text(text_origin, label, fill=color)
    

def one_image(img, predictions, min_score=0.2):
    img = Image.fromarray(img, mode='RGB')
    boxes = predictions[:, 1:5]
    classes = predictions[:, 6].astype(int)
    scores = predictions[:, 5]
    draw = ImageDraw.Draw(img)
    for idx in range(predictions.shape[0]):
        if scores[idx] < min_score:
            continue
        draw_detection(draw, boxes[idx], classes[idx])
    plt.figure(figsize=(80, 40))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

Run the inference on tensorflow:

In [ ]:
tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:
    tf.compat.v1.saved_model.load(sess, ['serve'], os.path.join(".", MODEL))
    detections = sess.run('detections:0', {"image_arrays:0": images})
    for batch in range(0, detections.shape[0]):
        one_image(images[batch], detections[batch])

And finally run the inference using onnxruntime:

In [6]:
import onnxruntime as rt
sess = rt.InferenceSession(MODEL + ".onnx")

In [8]:
detections = sess.run(['detections:0'], {"image_arrays:0": images})
start = time.time()
for _ in range(10):
    detections = sess.run(['detections:0'], {"image_arrays:0": images})
print((time.time()-start) / 10, "sec per inference")
detections = detections[0]
for batch in range(0, detections.shape[0]):
    one_image(images[batch], detections[batch])

0.4965421438217163 sec per inference
